# Tutorial: GPFA (Gaussian Process Factor Analysis)





- mention the original code and publication
- explain method

## Imports

In [1]:
cd ../..

/home/essink/working_environment/elephant


In [2]:
import elephant
from elephant.spike_train_generation import inhomogeneous_poisson_process
import quantities as pq
import neo

import numpy as np
from scipy.integrate import odeint

from elephant.gpfa import GPFA
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def _lorenz_ode(poi, t, s, r, b, tau):
    '''
    Given:
       poi: a point of interest in three dimensional space (tuple (x,y,z))
       t      : a point of interest in time
       s, r, b: parameters defining the lorenz attractor
    Returns:
       x_dot, y_dot, z_dot: values of the lorenz attractor's partial
           derivatives at the point x, y, z
    '''
    x, y, z = poi

    x_dot = (s*(y - x)) / tau
    y_dot = (r*x - y - x*z) / tau
    z_dot = (x*y - b*z) / tau
    return x_dot, y_dot, z_dot


def integrated_lorenz(dt, num_steps, x0=0, y0=1, z0=1.05,
                      s=10, r=28, b=2.667, tau=1e3):
    '''
    Given:
       dt        : integration time step in ms
       num_steps : number of integration steps -> max_time = dt*(num_steps-1)
       x0, y0, z0: initial values in three dimensional space
       s, r, b   : parameters defining the lorenz attractor
       tau       : characteristic timescale in ms
    Returns:
       t, (x, y, z): integrated trajectory of the Lorenz attractor
    '''
    assert isinstance(num_steps, int), "num_steps has to be integer"
    t = dt*np.arange(num_steps)
    poi = (x0, y0, z0)
    return t, odeint(_lorenz_ode, poi, t, args=(s, r, b, tau)).T


def integrated_oscillator(dt, num_steps, x0=0, y0=1, w=2*np.pi*1e-3):
    '''
    Given:
       dt       : integration time step in ms
       num_steps: number of integration steps -> max_time = dt*(num_steps-1)
       x0, y0   : initial values in two dimensional space
       w        : angular frequency in 1/ms
    Returns:
       t, (x, y): integrated trajectory of a harmonic oscillator
    '''
    assert isinstance(num_steps, int), "num_steps has to be integer"
    t = dt*np.arange(num_steps)
    x = x0*np.cos(w*t) + y0*np.sin(w*t)
    y = -x0*np.sin(w*t) + y0*np.cos(w*t)
    return t, np.array((x, y))

def random_projection(X, dim, loc=0, scale=None):
    '''
    Given:
       X    : data to embed, shape=(M, N)
       dim  : embedding dimension
       loc  : mean of random projection matrix
       scale: standard deviation of random projection matrix
    Returns:
       Y: random (normal) projection of input data, shape=(dim, N)
    '''
    if scale is None:
        scale = 1 / np.sqrt(X.shape[0])
    M = np.random.normal(loc, scale, (dim, X.shape[0]))
    return np.dot(M, X)



# Generate spike data

- start from a lorenz attractor
- randomly project lorenz attractor into higher dimensions
- use high dimensional trajectories as instantaneous rate to generate inhomogeneous spiketrains

In [61]:
dt = 1*pq.ms
transient_duration = 1*pq.s
trial_duration = 20*pq.s
num_steps_transient = int((transient_duration.rescale('ms')/dt).magnitude)
num_steps = int((trial_duration.rescale('ms')/dt).magnitude)
num_trials = 50
num_spiketrains = 20

#t, X = integrated_lorenz(dt, num_steps=num_steps_transient+num_steps, x0=0, y0=1, z0=1.25)
#Y = random_projection(X[:, num_steps_transient:], dim=num_spiketrains)
t, X = integrated_oscillator(dt.magnitude, num_steps=num_steps, x0=0, y0=1)
Y = random_projection(X, dim=num_spiketrains)

# Calculate instantaneous rate for in case of the oscillator
max_rate = 10*pq.Hz
Y_min = np.amin(Y)
Y_max = np.amax(Y)
norm_input = (Y - Y_min) / (Y_max - Y_min)
inst_rates = norm_input * max_rate

# Calculate instantaneous rate for in case of the Lorentz attractor
#inst_rates = np.power(20, Y)


In [53]:
f = plt.figure()
for i,y in enumerate(Y):
    plt.plot(y+i*np.max(Y))

In [63]:
for i,y in enumerate(inst_rates):
    plt.plot(y+i*np.max(inst_rates))

In [39]:
data = []
for _ in range(num_trials):
    spiketrains_per_trial = []
    for rate in inst_rates:
        inst_rate_anasig = neo.AnalogSignal(rate, sampling_rate=1/dt, units=pq.Hz)
        spiketrains_per_trial.append(inhomogeneous_poisson_process(inst_rate_anasig))
    data.append(spiketrains_per_trial)
t_max = t[-1] + t[1]

In [62]:
np.mean(inst_rates, axis=1)

array([ 9.7673215 , 15.19950844, 17.29229304, 16.29502546,  9.69342282,
        8.11990098, 13.41237011, 10.78291098,  9.45555061, 15.32154597,
        9.43264121,  9.2498299 , 11.13563896, 10.37952664, 17.64492335,
       10.21121025, 15.99666032, 10.46846245,  7.92603044, 10.15277356])

In [47]:
plt.eventplot(data[0])
#plt.xlim(0,1)

In [45]:
plt.eventplot(data[1])
plt.xlim(0,1)

(0, 1)

# Apply GPFA



In [9]:
%matplotlib qt

## 2D

In [40]:
gpfa = GPFA(x_dim=2)
xorth = gpfa.fit_transform(data)

EM iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Initializing parameters using factor analysis...

Fitting GPFA model...


EM iteration:  89%|████████▉ | 444/500 [01:02<00:08,  6.93it/s]

Fitting has converged after 445 EM iterations.)


In [41]:
f = plt.figure()
ax = f.gca()
lw = 0.5
col = 'k'
lw1 = 2
col1 = 'r'
alpha = 0.5

average_trajectories = np.zeros_like(xorth[0])
for i in range(num_trials):
    ax.plot(xorth[i][0], xorth[i][1], '-', linewidth=lw, color=col, alpha=alpha)
    average_trajectories += xorth[i]
average_trajectories /= num_trials

ax.plot(average_trajectories[0], average_trajectories[1], '-', linewidth=lw1, color=col1)

In [48]:
f1 = plt.figure()
ax = f1.gca()
lw = 1
col = 'k'

ax.plot(X[0], X[1], '.-', linewidth=lw, color=col)

## 3D

In [16]:
gpfa = GPFA(x_dim=3)
xorth = gpfa.fit_transform(data)

EM iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Initializing parameters using factor analysis...

Fitting GPFA model...


EM iteration: 100%|██████████| 500/500 [01:27<00:00,  5.40it/s]


In [21]:
%matplotlib qt

f = plt.figure()
ax = f.gca(projection='3d')
lw = 0.5
col = 'k'
lw1 = 2
col1 = 'r'
alpha = 0.5

average_trajectories = np.zeros_like(xorth[0])
for i in range(num_trials):
    ax.plot(xorth[i][0], xorth[i][1], xorth[i][2], '-', linewidth=lw, color=col, alpha=alpha)
    average_trajectories += xorth[i]
average_trajectories /= num_trials

ax.plot(average_trajectories[0], average_trajectories[1], average_trajectories[2], '-', linewidth=lw1, color=col1)
    


In [18]:
f1 = plt.figure()
ax = f1.gca(projection='3d')
lw = 1
col = 'k'

ax.plot(X[0], X[1], X[2], '.-', linewidth=lw, color=col)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
f2, (ax1,ax2,ax3) = plt.subplots(3,1)

times = inst_rate_anasig.times.rescale('s')
ax1.plot(times, X[0, num_steps_transient:])
ax2.plot(times, X[1, num_steps_transient:])
ax3.plot(times, X[2, num_steps_transient:])



In [ ]:
f3, (ax1,ax2,ax3) = plt.subplots(3,1)

times = np.arange(average_trajectories.shape[1])*0.02
ax1.plot(times, average_trajectories[0])
ax2.plot(times, average_trajectories[1])
ax3.plot(times, average_trajectories[2])

# Cross-validation

In [ ]:
lls = []
for x_dim in range(1, 6):
    gpfa = GPFA(x_dim=x_dim)
    lls.append(np.mean(cross_val_score(gpfa, data, cv=5)))

In [ ]:
plt.plot(np.arange(1,6), lls, '.-')